# Introduction to Classification (Part II)

In this tutorial, we extend our probabilistic interpretation from the [Part I](./regression.ipynb) of the tutorial to build a digits classifier. As earlier, we will come across more functions for numerical computation, and plotting.

### Common Imports

In [ ]:
import numpy as np
import scipy as sp
import matplotlib as plt
import seaborn as sns

sns.set(font_scale=1.25, style='whitegrid', palette=sns.color_palette("Set1"))

## Classfication Problem Setup

> For a set of $K$ classes, the goal of classification is to assign a class label $y \in \{0,\dots,K-1\}$ for every $d$-dimensional input variable (predictors) $\mathbf{x} \in \mathbb{R}^{d}$. In a typical setting, we will generally have $n$ training samples, which we will index by subscripts $[ \mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n ]$, with each having a corresponding target observed value, $[ y_1, y_2, \dots, y_n]$.

## The MNIST Dataset

The MNIST dataset is a classic dataset of grayscale images containing digits from 0 to 9. Our task will be train a classifier, such that given an image, we can predict the digit. Therefore, we consider each image as $\mathbf{x}$ and each class label as $y \in \{0, 1, \dots, 9\}$ (i.e. $K = 10$).

In machine learning, we typically split our dataset into a training set and a test set. By only learning with the training set, and testing on an unseen test set, we can estimate the performance of our learned classifier when actually used in the real-world.

### Loading the dataset

We have the MNIST dataset stored as `numpy` zip-arrays, and use the [`np.load`](https://numpy.org/doc/stable/reference/generated/numpy.load.html) method to them below.

**NOTE**: Datasets may be stored in various formats, and may often require ad-hoc routines to load and prepare for training. In this case, we keep things simple for ourselves.

In [ ]:
train_data = np.load('mnist_train_uint8.npz', allow_pickle=True)
train_X, train_y = train_data['X'], train_data['y']

test_data = np.load('mnist_test_uint8.npz', allow_pickle=True)
test_X, test_y = test_data['X'], test_data['y']

assert train_X.shape[0] == train_y.shape[0]
assert test_X.shape[0] == test_y.shape[0]

N_train = train_X.shape[0]
N_test = test_X.shape[0]

print(f'Training set size: {N_train}, Test set size: {N_test}')

train_X.shape, train_y.shape

Each image is a grayscale image, represented as a $28 \times 28$ array, where the values range from $0$ (black) to $255$ (white).

## Logistic Regression

We will extend our probabilistic interpretation from [Part I](./regression.ipynb). This time we posit the same linear model, written in vectorized form with parameters $\theta$ and inputs $\mathbf{x}$ augmented with 1 to account for the intercept term. But we realize, that now we are predicting not continuous values, but categorical values. Therefore, we posit that the $\log$-probability of an output class $c$ is given by the vector product

$$
\log{p\left( y_i = c \mid \mathbf{x} \right)} = \begin{bmatrix}\cdots & \mathbf{x} & \cdots & 1\end{bmatrix}\begin{bmatrix}\theta_{c,1} \\ \vdots \\ \theta_{c,d} \\ \theta_{c,d+1} \end{bmatrix}
$$

The important bit to note here is that for every class, we have a separate set of parameters. Thus, the total number of parameters we will be learning is $K \times (d + 1)$, where the extra $1$ factor comes from handling the intercept term. This term is often known as the _bias_. The objective again is to maximize the $\log$-likelihood, as we have done earlier.

The "regression" in its name is an unfortunate misnomer. Nevertheless, _logistic regression_ is a direct extension of _linear regression_, appropriated for a discrete probability setting. Note that we ensure the non-negativity of the probabilty values by simply exponentiating the term.

**TIP**: There are other function transforms to ensure positivity of the inner product we have considered above. For instance, [Probit Regression](https://en.wikipedia.org/wiki/Probit_model) uses the cumulative density function (CDF) of the Gaussian distribution we've seen in [Part I](./regression.ipynb). Any appropriate transform is valid in general, but may come at the cost of making the optimization harder.

## Training an MNIST Classifier

In our formulation above, we have considered $\mathbf{x}$ to be a vector. Our `train_X` and `test_X` are, however, a set of $28 \times 28$ images. We make a simple transform by simply flattening these images into a big $784$-dimensional vector, and use that as our input. This what the `.reshape()` operation does below. Further, to make the optimization problem better-conditioned, we scale all values to be between $0.0$ and $1.0$, by dividing by $255$ which is the maximum possible pixel value.

In [ ]:
train_X = train_X.reshape(N_train, -1).astype(np.float) / 255
test_X = test_X.reshape(N_test, -1).astype(np.float) / 255

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_y);

### Training Accuracy

In [ ]:
model.score(train_X, train_y)

### Test Accuracy

In [ ]:
model.score(test_X, test_y)

### Confusion Matrix

A common way to visualize the performance of a classifier is the plot the confusion matrix. A confusion matrix $\mathcal{C}$ is a $K \times K$ matrix, such that each element $\mathcal{C}_{i,j}$ represents the total number of inputs which belong to class $i$, but were predicted to be in class $j$.

We use the [`sns.heatmap`](http://seaborn.pydata.org/generated/seaborn.heatmap.html) to plot this matrix.

In [ ]:
from sklearn.metrics import confusion_matrix

cmat = confusion_matrix(model.predict(test_X.reshape(N_test, -1)), test_y)

# print(cmat)

g = sns.heatmap(cmat)
g.set_xlabel('Predicted Class')
g.set_ylabel('True Class');